In [1]:
!pip install langchain langchain-openai

import os
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.2 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain

## Simple LLM Chain

In [5]:
# load in chat model
chat = ChatOpenAI(temperature=0.0)

# create prompt template
prompt = ChatPromptTemplate.from_template(
    "Translate this text to french: {text}"
)

# create LLM Chain
chain = LLMChain(llm=chat, prompt=prompt)

# run the chain
chain.invoke("Can you translate this sentence?")

{'text': 'Pouvez-vous traduire cette phrase ?'}

## Simple Sequential Chain

In [7]:
chain_two_prompt = ChatPromptTemplate.from_template(
    "Translate this text to spanish: {french_text}"
)

chain_three_prompt = ChatPromptTemplate.from_template(
    "Translate this text to english: {spanish_text}"
)

chain_one = LLMChain(llm=chat, prompt=prompt)
chain_two = LLMChain(llm=chat, prompt=chain_two_prompt)
chain_three = LLMChain(llm=chat, prompt=chain_three_prompt)

simple_sequential_chain = SimpleSequentialChain(chains=[chain_one, chain_two, chain_three],
                                                verbose=True)

simple_sequential_chain.invoke("Can you translate this sentence?")



> Entering new SimpleSequentialChain chain...
Pouvez-vous traduire cette phrase ?
¿Puedes traducir esta frase?
Can you translate this sentence?

> Finished chain.


{'input': 'Can you translate this sentence?',
 'output': 'Can you translate this sentence?'}

## Advanced Sequential Chain

In [9]:
chain_one = LLMChain(llm=chat, prompt=prompt, output_key="french_text")
chain_two = LLMChain(llm=chat, prompt=chain_two_prompt, output_key="spanish_text")
chain_three = LLMChain(llm=chat, prompt=chain_three_prompt, output_key="english_text")


chain_four_prompt = ChatPromptTemplate.from_template(
    """Answer 'Yes' if the following two sentences are the exact same, else answer 'No':

    Sentence 1: {text}
    Sentence 2: {english_text}
    """
)

chain_four = LLMChain(llm=chat, prompt=chain_four_prompt, output_key="evaluation")

adv_sequential_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["text"],
    output_variables=["text", "english_text","evaluation"],
    verbose=True
)

adv_sequential_chain.invoke("Can you translate this sentence?")



> Entering new SequentialChain chain...

> Finished chain.


{'text': 'Can you translate this sentence?',
 'english_text': 'Can you translate this sentence?',
 'evaluation': 'Yes'}